In [16]:
# Import neccessary libraries
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
from skspatial.objects import Line

In [17]:
# Create a VideoCapture object
cap = cv2.VideoCapture('video_z_auta.mp4')

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Convert to gray scale and improve by equalization histogram
    gray_first = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray_first)
    
    # Dimensions of the video-frame
    row, coll = gray.shape
    # Apply the Canny 
    edges = cv2.Canny(gray, threshold1= 100, threshold2 = 150)
    # Dimensions of "vision belt" - part of the frame we need for line detection
    y_up = 473
    y_down = 600
    # Masked everything out of the "vision belt"
    edges[0:y_up] = 0
    edges[y_down:row] = 0
    
    # create a zero array from edges for additional masking
    zero_edges = np.zeros_like(edges)

    # Create the polygon using 4 points for additional masking the frame. 
    polygon = np.array([[133,600], [467,467], [690,467], [907,600]])
    
    
    # Create a mask from polygon 
    mask = cv2.fillConvexPoly(zero_edges, polygon, 1)
    # Create another polygon for masking the central part of lane.
    polygon2 = np.array([[400,600], [550,473], [600,473], [700,600]])
    
    mask2 = cv2.fillConvexPoly(mask, polygon2, 0)
    # Apply the polygon on the frame 
    edges_masked = cv2.bitwise_and(edges, edges, mask=mask2)
    
    # Apply the HoughLinesP function for founding the lines from Canny.
    lines = []
    lines = cv2.HoughLinesP(edges_masked, 1, np.pi/90, 30 ,maxLineGap=200)
    
    # Condition if the HoughLinesP function is failed - do not show any lines. 
    if lines is None:
        result = frame
        cv2.putText(result, 
                    'NO TYPE', 
                    (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1, 
                    (0, 255, 255), 
                    2, 
                    cv2.LINE_4)
    
    # Condition if there are too many details - do not show any lines.    
    else:
        if len(lines)>20:
            result = frame
            too_many_lines_str = 'TOO MANY DETAILS - ' + str(len(lines))
            cv2.putText(result, 
                too_many_lines_str, 
                (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)
        else:
            # Calculate k of each lines  (y=kx+b)
            k = []
            for i in range (len(lines)):
                x1, y1, x2, y2 = lines[i][0]
                k.append((y2-y1)/((x2-x1)+1))   # +1 for avoiding zero dividing      

            # Calculate b of each lines  (y=kx+b)
            b = []
            for i in range(len(lines)):
                x1,y1,x2,y2 = lines[i][0]
                b.append(y1-k[i]*x1)     

            # Create  dictinaries for sorting left and right lines
            k_and_b = dict(zip(k,b))
            

            # Create two lists of k for left and right lines
            left_k = []
            right_k = []

            # Cycle for sorting left and right k. If k is more or less than ...
            for i in k:
                if i<-0.3:
                    left_k.append(i)
                else: 
                    if i>0.3:
                        right_k.append(i)

            # Condition if both lists are empty  - do not show any lines.
            if not left_k or not right_k:
                result = frame

            else:
                # Create list of left b and dict for left lines using dict k_and_b we made before.
                left_b = []
                left_lines = {}
                for i in left_k:
                    left_b.append(k_and_b[i])
                    left_lines = dict(zip(left_b, left_k))
                    
                # Create list of right b and dict for right lines using dict k_and_b we made before.
                right_b = []
                right_lines = {}
                for i in right_k:
                    right_b.append(k_and_b[i])
                    right_lines = dict(zip(right_b, right_k))
                
                # Keep only one line from the left and one line from the right side. 
                # Closest left line has max b, closest right line has min b. 
                
                result_b_left = max(left_lines.keys())
                result_k_left = left_lines[max(left_lines.keys())]

                result_b_right = min(right_lines.keys())
                result_k_right = right_lines[min(right_lines.keys())]
                   
                

                # Calc the two points X coordinates for left and right line. 
                # As Y coordinates we use the "vision belt" up and down values. 

                x1_left = int((y_up-result_b_left)/result_k_left)
                x2_left = int((y_down-result_b_left)/result_k_left)

                x1_right = int((y_up-result_b_right)/result_k_right)
                x2_right = int((y_down-result_b_right)/result_k_right)

                
               
                # Make a copy of the video frame
                result = np.copy(frame)
                # Draw the left and right lines on the videoframe. 

                result = cv2.line(result, (x1_left,y_up), (x2_left,y_down), (0,255,0), 3, cv2.LINE_AA)
                result = cv2.line(result, (x1_right,y_up), (x2_right,y_down), (0,255,0), 3, cv2.LINE_AA)  
                result_k_left_str = 'K-LEFT - ' + str(result_k_left)
                result_k_right_str = 'K-RIGHT - ' + str(result_k_right)
                cv2.putText(result, 
                result_k_left_str, 
                (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)

                cv2.putText(result, 
                result_k_right_str, 
                (50, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)

                lines_str = 'TOTAL DETECTED LINES - ' + str(len(lines))
                cv2.putText(result, 
                lines_str, 
                (50, 150), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 255, 255), 
                2, 
                cv2.LINE_4)
    if ret == True:
    # Display the resulting frame
        cv2.imshow('Frame',result)

    # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

  # Break the loop
    else: 
        break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()



qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 14018, resource id: 0, major code: 140 (Unknown), minor code: 20
qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 14028, resource id: 0, major code: 140 (Unknown), minor code: 20
qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 14117, resource id: 0, major code: 140 (Unknown), minor code: 20
